In [1]:
import pandas as pd
import numpy as np
import warnings
import seaborn as sns
from matplotlib.pyplot import figure
from catboost import CatBoostRegressor

warnings.filterwarnings('ignore')

RANDOM_SEED = 41


In [2]:
def update_produv(produv_train):
    
    produv_start=pd.pivot_table(produv_train,index='NPLV',values='SEC', aggfunc={'min'}).reset_index()
    produv_end=pd.pivot_table(produv_train,index='NPLV',values='SEC', aggfunc={'max'}).reset_index()
    ras_med=pd.pivot_table(produv_train,index='NPLV',values='RAS', aggfunc={'mean'}).reset_index()
    ras_sum=pd.pivot_table(produv_train,index='NPLV',values='RAS', aggfunc={'sum'}).reset_index()
    pol_start=pd.pivot_table(produv_train,index='NPLV',values='POL', aggfunc={'max'}).reset_index()
    pol_end=pd.pivot_table(produv_train,index='NPLV',values='POL', aggfunc={'min'}).reset_index()
    
    produv_upd=pd.merge(produv_start,produv_end, how='left',on='NPLV')
    produv_upd=pd.merge(produv_upd,ras_sum, how='left',on='NPLV')
    produv_upd=pd.merge(produv_upd,pol_start, how='left',on='NPLV')
    produv_upd=pd.merge(produv_upd,pol_end, how='left',on='NPLV')
    
    #df.rename(columns = {list(df)[1]:'new_name'}, inplace=True)
    
    produv_upd.rename(columns={list(produv_upd)[1]: 'PRODUVKA_START'}, inplace=True)
    produv_upd.rename(columns={list(produv_upd)[2]: 'PRODUVKA_END'}, inplace=True)
    produv_upd.rename(columns={list(produv_upd)[3]: 'RASHOD_SUM'}, inplace=True)
    produv_upd.rename(columns={list(produv_upd)[4]: 'POL_MAX'}, inplace=True)
    produv_upd.rename(columns={list(produv_upd)[5]: 'POL_MIN'}, inplace=True)
    
    produv_upd['PRODUVKA_TIME']=produv_upd['PRODUVKA_END']-produv_upd['PRODUVKA_START']
    produv_upd['PRODUVKA_TIME,SEC']=produv_upd['PRODUVKA_TIME'].dt.seconds

    
    return produv_upd

In [3]:
def update_gas(gas_train):
    V_finish=gas_train[['NPLV','Time','V']].sort_values(by=['NPLV','Time'], ascending=[True,False]).drop_duplicates('NPLV')
    V_finish=V_finish.drop('Time', axis=1)
    V_start=gas_train[['NPLV','Time','V']].sort_values(by=['NPLV','Time'], ascending=[True,True]).drop_duplicates('NPLV')
    V_start=V_start.drop('Time', axis=1)
    T_finish=gas_train[['NPLV','Time','T']].sort_values(by=['NPLV','Time'], ascending=[True,False]).drop_duplicates('NPLV')
    T_finish=T_finish.drop('Time', axis=1)
    T_start=gas_train[['NPLV','Time','T']].sort_values(by=['NPLV','Time'], ascending=[True,True]).drop_duplicates('NPLV')
    T_start=T_start.drop('Time', axis=1)
    O2_sum=pd.pivot_table(gas_train,index='NPLV',values=['O2'], aggfunc={'sum'}).reset_index()
    N2_sum=pd.pivot_table(gas_train,index='NPLV',values=['N2'], aggfunc={'sum'}).reset_index()
    H2_sum=pd.pivot_table(gas_train,index='NPLV',values='H2', aggfunc={'sum'}).reset_index()
    CO2_sum=pd.pivot_table(gas_train,index='NPLV',values=['CO2'], aggfunc={'sum'}).reset_index()
    CO_sum=pd.pivot_table(gas_train,index='NPLV',values=['CO'], aggfunc={'sum'}).reset_index()
    AR_sum=pd.pivot_table(gas_train,index='NPLV',values=['AR'], aggfunc={'sum'}).reset_index()
    TF1_start=gas_train[['NPLV','Time','T фурмы 1']].sort_values(by=['NPLV','Time'], ascending=[True,True]).drop_duplicates('NPLV')
    TF1_start=TF1_start.drop('Time', axis=1)
    TF1_end=gas_train[['NPLV','Time','T фурмы 1']].sort_values(by=['NPLV','Time'], ascending=[True,False]).drop_duplicates('NPLV')
    TF1_end=TF1_end.drop('Time', axis=1)
    TF2_start=gas_train[['NPLV','Time','T фурмы 2']].sort_values(by=['NPLV','Time'], ascending=[True,True]).drop_duplicates('NPLV')
    TF2_start=TF2_start.drop('Time', axis=1)
    TF2_end=gas_train[['NPLV','Time','T фурмы 2']].sort_values(by=['NPLV','Time'], ascending=[True,False]).drop_duplicates('NPLV')
    TF2_end=TF2_end.drop('Time', axis=1)
    O2_pressure_start=gas_train[['NPLV','Time','O2_pressure']].sort_values(by=['NPLV','Time'], ascending=[True,True]).drop_duplicates('NPLV')
    O2_pressure_start=O2_pressure_start.drop('Time', axis=1)
    O2_pressure_end=gas_train[['NPLV','Time','O2_pressure']].sort_values(by=['NPLV','Time'], ascending=[True,False]).drop_duplicates('NPLV')
    O2_pressure_end=O2_pressure_end.drop('Time', axis=1)
    O2_pressure_max=pd.pivot_table(gas_train,index='NPLV',values='O2_pressure', aggfunc={'max'}).reset_index()
    O2_pressure_min=pd.pivot_table(gas_train,index='NPLV',values='O2_pressure', aggfunc={'min'}).reset_index()
    time_start=pd.pivot_table(gas_train,index='NPLV',values='Time', aggfunc={'min'}).reset_index()
    time_end=pd.pivot_table(gas_train,index='NPLV',values='Time', aggfunc={'max'}).reset_index()
    tmp_1=pd.merge(time_start, time_end, how='left', on='NPLV')
    tmp_1=pd.merge(tmp_1, V_start, how='left', on='NPLV')
    tmp_1=pd.merge(tmp_1, V_finish, how='left', on='NPLV')
    tmp_1=pd.merge(tmp_1, T_start, how='left', on='NPLV')
    tmp_1=pd.merge(tmp_1, T_finish, how='left', on='NPLV')
    tmp_1=pd.merge(tmp_1, O2_sum, how='left', on='NPLV')
    tmp_1=pd.merge(tmp_1, N2_sum, how='left', on='NPLV')
    tmp_1=pd.merge(tmp_1, H2_sum, how='left', on='NPLV')
    tmp_1=pd.merge(tmp_1, CO2_sum, how='left', on='NPLV')
    tmp_1=pd.merge(tmp_1, CO_sum, how='left', on='NPLV')
    tmp_1=pd.merge(tmp_1, AR_sum, how='left', on='NPLV')
    tmp_1=pd.merge(tmp_1, TF1_start, how='left', on='NPLV')
    tmp_1=pd.merge(tmp_1, TF1_end, how='left', on='NPLV')
    tmp_1=pd.merge(tmp_1, TF2_start, how='left', on='NPLV')
    tmp_1=pd.merge(tmp_1, TF2_end, how='left', on='NPLV')
    tmp_1=pd.merge(tmp_1, O2_pressure_start, how='left', on='NPLV')
    tmp_1=pd.merge(tmp_1, O2_pressure_end, how='left', on='NPLV')
    tmp_1=pd.merge(tmp_1, O2_pressure_max, how='left', on='NPLV')
    tmp_1=pd.merge(tmp_1, O2_pressure_min, how='left', on='NPLV')
    
    
    tmp_1.rename(columns={list(tmp_1)[1]:'GAS_START'},inplace=True)
    tmp_1.rename(columns={list(tmp_1)[2]: 'GAS_FINISH'}, inplace=True)
    tmp_1.rename(columns={list(tmp_1)[3]: 'V_START'}, inplace=True)
    tmp_1.rename(columns={list(tmp_1)[4]: 'V_FINISH'}, inplace=True)
    tmp_1.rename(columns={list(tmp_1)[5]: 'T_START'}, inplace=True)
    tmp_1.rename(columns={list(tmp_1)[6]: 'T_FINISH'}, inplace=True)
    tmp_1.rename(columns={list(tmp_1)[7]: 'O2_SUM'}, inplace=True)
    tmp_1.rename(columns={list(tmp_1)[8]: 'N2_SUM'}, inplace=True)
    tmp_1.rename(columns={list(tmp_1)[9]: 'H2_SUM'}, inplace=True)
    tmp_1.rename(columns={list(tmp_1)[10]: 'CO2_SUM'}, inplace=True)
    tmp_1.rename(columns={list(tmp_1)[11]: 'CO_SUM'}, inplace=True)
    tmp_1.rename(columns={list(tmp_1)[12]: 'AR_SUM'}, inplace=True)
    tmp_1.rename(columns={list(tmp_1)[13]: 'TF1_START'}, inplace=True)
    tmp_1.rename(columns={list(tmp_1)[14]: 'TF1_FINISH'}, inplace=True)
    tmp_1.rename(columns={list(tmp_1)[15]: 'TF2_START'}, inplace=True)
    tmp_1.rename(columns={list(tmp_1)[16]: 'TF2_FINISH'}, inplace=True)
    tmp_1.rename(columns={list(tmp_1)[17]: 'O2_PRESSURE_START'}, inplace=True)
    tmp_1.rename(columns={list(tmp_1)[18]: 'O2_PRESSURE_FINISH'}, inplace=True)
    tmp_1.rename(columns={list(tmp_1)[19]: 'O2_PRESSURE_MAX'}, inplace=True)
    tmp_1.rename(columns={list(tmp_1)[20]: 'O2_PRESSURE_MIN'}, inplace=True)
    
    tmp_1['GAS_TIME']=tmp_1['GAS_FINISH']-tmp_1['GAS_START']
    tmp_1['GAS_TIME,SEC']=tmp_1['GAS_TIME'].dt.seconds
    
    tmp_1=tmp_1.drop('GAS_TIME', axis=1)
    tmp_1=tmp_1.drop('GAS_FINISH', axis=1)
    
    
    return tmp_1
    

In [4]:
def update_sip(sip_train):
    sip_time_start=sip_train[['NPLV','DAT_OTD']].sort_values(by=['NPLV','DAT_OTD'], ascending=[True,True]).drop_duplicates('NPLV')
    sip_time_finish=sip_train[['NPLV','DAT_OTD']].sort_values(by=['NPLV','DAT_OTD'], ascending=[True,False]).drop_duplicates('NPLV')
    sip_sostav=pd.pivot_table(sip_train,index='NPLV',columns='VDSYP',values='VSSYP', aggfunc={'sum'})
    tmp_1=pd.merge(sip_sostav, sip_time_start, how='left',on='NPLV')
    tmp_1=pd.merge(tmp_1, sip_time_finish, how='left',on='NPLV')
    tmp_1.rename(columns={list(tmp_1)[1]:'TOTAL_104'}, inplace=True)
    tmp_1.rename(columns={list(tmp_1)[2]:'TOTAL_119'}, inplace=True)
    tmp_1.rename(columns={list(tmp_1)[3]:'TOTAL_171'}, inplace=True)
    tmp_1.rename(columns={list(tmp_1)[4]:'TOTAL_346'}, inplace=True)
    tmp_1.rename(columns={list(tmp_1)[5]:'TOTAL_397'}, inplace=True)
    tmp_1.rename(columns={list(tmp_1)[6]:'TOTAL_408'}, inplace=True)
    tmp_1.rename(columns={list(tmp_1)[7]:'TOTAL_442'}, inplace=True)
    tmp_1.rename(columns={list(tmp_1)[8]:'SIP_START'}, inplace=True)
    tmp_1.rename(columns={list(tmp_1)[9]:'SIP_FINISH'}, inplace=True)
    tmp_1['SIP_TIME']=tmp_1['SIP_FINISH']-tmp_1['SIP_START']
    tmp_1['SIP_TIME,SEC']=tmp_1['SIP_TIME'].dt.seconds
    
    tmp_1=tmp_1.drop('SIP_TIME', axis=1)
    tmp_1=tmp_1.drop('SIP_FINISH', axis=1)
    
    return tmp_1

In [5]:
def update_chronom(df):
    
    O2_sum=pd.pivot_table(df,index='NPLV',values='O2', aggfunc={'sum'}).reset_index()
        
    O2_sum.rename(columns={list(O2_sum)[1]: 'O2_SUM_CHR'}, inplace=True)      
   
    return O2_sum

In [6]:
def update_lom(lom):
    
    vdl_sum=pd.pivot_table(lom,index='NPLV',values='VDL', aggfunc={'sum'}).reset_index()
    lom_sum=pd.pivot_table(lom,index='NPLV',values='VES', aggfunc={'sum'}).reset_index()
    
    lom_upd=pd.merge(vdl_sum, lom_sum, how='left',on='NPLV')
        
    #df.rename(columns = {list(df)[1]:'new_name'}, inplace=True)
    
    lom_upd.rename(columns={list(lom_upd)[1]: 'VDL_SUM'}, inplace=True)
    lom_upd.rename(columns={list(lom_upd)[2]: 'LOM_SUM'}, inplace=True)
        
    return lom_upd

In [7]:
path='D:/data/'

names=['chronom','chugun','gas','lom','plavki','produv','sip']

chrom_train=pd.read_csv(path+names[0]+'_train.csv', )
chugun_train=pd.read_csv(path+names[1]+'_train.csv')
gas_train=pd.read_csv(path+names[2]+'_train.csv')
lom_train=pd.read_csv(path+names[3]+'_train.csv')
plavki_train=pd.read_csv(path+names[4]+'_train.csv')
produv_train=pd.read_csv(path+names[5]+'_train.csv')
sip_train=pd.read_csv(path+names[6]+'_train.csv')

chrom_test=pd.read_csv(path+names[0]+'_test.csv')
chugun_test=pd.read_csv(path+names[1]+'_test.csv')
gas_test=pd.read_csv(path+names[2]+'_test.csv')
lom_test=pd.read_csv(path+names[3]+'_test.csv')
plavki_test=pd.read_csv(path+names[4]+'_test.csv')
produv_test=pd.read_csv(path+names[5]+'_test.csv')
sip_test=pd.read_csv(path+names[6]+'_test.csv')

target_train=pd.read_csv(path+'target_train.csv')
   

In [8]:
chrom_train.head()

,Unnamed: 0,NPLV,TYPE_OPER,NOP,VR_NACH,VR_KON,O2
0,35171,510008,межпл.прост.,межпл. простой,2011-01-01 10:28:13,2011-01-01 10:32:15,NaN
1,35162,510008,межпл.прост.,межпл. простой,2021-01-01 03:01:07,2021-01-01 03:08:11,NaN
2,35163,510008,межпл.прост.,Осмотр конвертера,2021-01-01 03:01:19,2021-01-01 03:03:43,NaN
3,35164,510008,межпл.прост.,Наведение гарнисажа,2021-01-01 03:03:43,2021-01-01 03:05:23,NaN
4,35165,510008,межпл.прост.,Слив шлака,2021-01-01 03:05:23,2021-01-01 03:06:09,NaN


In [9]:
chrom_train=chrom_train[chrom_train['VR_NACH']>'2012-01-01 10:28:13']


In [10]:
chrom_train['VR_NACH']=pd.to_datetime(chrom_train['VR_NACH'], format='%Y-%m-%d %H:%M:%S')
chrom_train['VR_KON']=pd.to_datetime(chrom_train['VR_KON'], format='%Y-%m-%d %H:%M:%S')

chugun_train['DATA_ZAMERA']=pd.to_datetime(chugun_train['DATA_ZAMERA'], format='%Y-%m-%d %H:%M:%S')

In [11]:
chrom_train

,Unnamed: 0,NPLV,TYPE_OPER,NOP,VR_NACH,VR_KON,O2
1,35162,510008,межпл.прост.,межпл. простой,2021-01-01 03:01:07,2021-01-01 03:08:11,NaN
2,35163,510008,межпл.прост.,Осмотр конвертера,2021-01-01 03:01:19,2021-01-01 03:03:43,NaN
3,35164,510008,межпл.прост.,Наведение гарнисажа,2021-01-01 03:03:43,2021-01-01 03:05:23,NaN
4,35165,510008,межпл.прост.,Слив шлака,2021-01-01 03:05:23,2021-01-01 03:06:09,NaN
5,35166,510008,межпл.прост.,Отсутствие O2,2021-01-01 03:06:09,2021-01-01 03:08:15,NaN
...,...,...,...,...,...,...,...
34401,20943,512322,опер,Повалка,2021-04-26 18:31:26,2021-04-26 18:41:10,NaN
34402,20944,512322,опер,Выпуск металла,2021-04-26 18:41:10,2021-04-26 18:47:40,NaN
34403,20945,512322,опер,Отсечка шлака,2021-04-26 18:46:15,2021-04-26 18:46:15,NaN
34404,20946,512322,опер,Прерыв. выпуска,2021-04-26 18:47:40,2021-04-26 18:48:40,NaN


In [12]:
produv_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4729802 entries, 0 to 4729801
Data columns (total 4 columns):
 #   Column  Dtype  
---  ------  -----  
 0   NPLV    int64  
 1   SEC     object 
 2   RAS     float64
 3   POL     float64
dtypes: float64(2), int64(1), object(1)
memory usage: 144.3+ MB


In [13]:
produv_train.head(10)

,NPLV,SEC,RAS,POL
0,510008,2021-01-01 03:18:26,382.00,3.92
1,510008,2021-01-01 03:18:28,382.00,3.92
2,510008,2021-01-01 03:18:30,553.00,3.92
3,510008,2021-01-01 03:18:32,701.00,3.92
4,510008,2021-01-01 03:18:34,813.00,3.92
5,510008,2021-01-01 03:18:36,888.00,3.92
6,510008,2021-01-01 03:18:38,939.00,3.45
7,510008,2021-01-01 03:18:40,965.00,2.87
8,510008,2021-01-01 03:18:42,962.45,2.87
9,510008,2021-01-01 03:18:44,959.90,2.87


In [14]:
produv_train['SEC']=pd.to_datetime(produv_train['SEC'], format='%Y-%m-%d %H:%M:%S')

In [15]:
produv_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4729802 entries, 0 to 4729801
Data columns (total 4 columns):
 #   Column  Dtype         
---  ------  -----         
 0   NPLV    int64         
 1   SEC     datetime64[ns]
 2   RAS     float64       
 3   POL     float64       
dtypes: datetime64[ns](1), float64(2), int64(1)
memory usage: 144.3 MB


In [16]:
produv_upd=update_produv(produv_train)

In [17]:
produv_upd

,NPLV,PRODUVKA_START,PRODUVKA_END,RASHOD_SUM,POL_MAX,POL_MIN,PRODUVKA_TIME,"PRODUVKA_TIME,SEC"
0,510008,2021-01-01 03:18:26,2021-01-01 04:26:54,838814.0,7.070000,0.77,0 days 01:08:28,4108
1,510009,2021-01-01 04:26:56,2021-01-01 05:25:08,764750.0,4.760000,0.63,0 days 00:58:12,3492
2,510010,2021-01-01 05:25:10,2021-01-01 06:23:50,764170.5,4.349765,0.65,0 days 00:58:40,3520
3,510011,2021-01-01 06:23:52,2021-01-01 07:26:04,733717.0,12.490000,0.79,0 days 01:02:12,3732
4,510012,2021-01-01 07:26:06,2021-01-01 08:36:44,840471.0,5.000000,0.67,0 days 01:10:38,4238
...,...,...,...,...,...,...,...,...
2058,512318,2021-04-26 13:10:12,2021-04-26 14:37:38,978520.0,10.100000,0.80,0 days 01:27:26,5246
2059,512319,2021-04-26 14:37:40,2021-04-26 15:35:46,783447.0,10.040000,0.83,0 days 00:58:06,3486
2060,512320,2021-04-26 15:35:48,2021-04-26 16:36:02,742971.0,7.270000,0.90,0 days 01:00:14,3614
2061,512321,2021-04-26 16:36:04,2021-04-26 18:08:46,964755.5,10.580000,0.92,0 days 01:32:42,5562


In [18]:
gas_train.head()

,NPLV,Time,V,T,O2,N2,H2,CO2,CO,AR,T фурмы 1,T фурмы 2,O2_pressure
0,510008,2021-01-01 03:08:11.437,218263.343750,262.847229,18.722993,80.132247,0.087755,0.163878,0.009229,0.893243,0.0,0.0,13.085938
1,510008,2021-01-01 03:08:12.437,218263.343750,262.847229,18.732721,80.138406,0.087959,0.148980,0.008390,0.892948,0.0,0.0,13.085938
2,510008,2021-01-01 03:08:13.437,218369.359375,262.152771,18.742449,80.144565,0.088163,0.134082,0.007551,0.892653,0.0,0.0,13.085938
3,510008,2021-01-01 03:08:14.437,218475.359375,261.805573,18.752177,80.150724,0.088367,0.119184,0.006712,0.892358,0.0,0.0,13.093172
4,510008,2021-01-01 03:08:15.437,218369.359375,260.763885,18.761905,80.156883,0.088571,0.104286,0.005873,0.892063,0.0,0.0,13.093172


In [19]:
gas_train['Time']=pd.to_datetime(gas_train['Time'], format='%Y-%m-%d %H:%M:%S')

In [20]:
gas_upd=update_gas(gas_train)

In [21]:
gas_upd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2063 entries, 0 to 2062
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   NPLV                2063 non-null   int64         
 1   GAS_START           2063 non-null   datetime64[ns]
 2   V_START             2063 non-null   float64       
 3   V_FINISH            2063 non-null   float64       
 4   T_START             2063 non-null   float64       
 5   T_FINISH            2063 non-null   float64       
 6   O2_SUM              2063 non-null   float64       
 7   N2_SUM              2063 non-null   float64       
 8   H2_SUM              2063 non-null   float64       
 9   CO2_SUM             2063 non-null   float64       
 10  CO_SUM              2063 non-null   float64       
 11  AR_SUM              2063 non-null   float64       
 12  TF1_START           2063 non-null   float64       
 13  TF1_FINISH          2063 non-null   float64     

In [22]:
sip_train.head()

,NPLV,VDSYP,NMSYP,VSSYP,DAT_OTD
0,510008,346,Уголь ТО,570,2021-01-01 03:03:53
1,510008,346,Уголь ТО,220,2021-01-01 03:04:10
2,510008,408,изв_ЦОИ,7300,2021-01-01 03:08:17
3,510008,346,Уголь ТО,270,2021-01-01 03:09:03
4,510008,346,Уголь ТО,430,2021-01-01 03:09:20


In [23]:
sip_train['DAT_OTD']=pd.to_datetime(sip_train['DAT_OTD'], format='%Y-%m-%d %H:%M:%S')

In [24]:
sip_upd=update_sip(sip_train)

In [25]:
sip_upd.head()

,NPLV,TOTAL_104,TOTAL_119,TOTAL_171,TOTAL_346,TOTAL_397,TOTAL_408,TOTAL_442,SIP_START,"SIP_TIME,SEC"
0,510008,NaN,NaN,980.0,2950.0,NaN,14080.0,2960.0,2021-01-01 03:03:53,1795
1,510009,1060.0,NaN,960.0,2930.0,NaN,18830.0,NaN,2021-01-01 03:55:43,2670
2,510010,990.0,NaN,1050.0,2990.0,NaN,16080.0,2960.0,2021-01-01 05:09:20,1892
3,510011,550.0,NaN,980.0,3620.0,NaN,22150.0,NaN,2021-01-01 06:01:36,3304
4,510012,2050.0,NaN,1000.0,2930.0,NaN,19550.0,3010.0,2021-01-01 07:11:44,2315


In [26]:
chronom_upd=update_chronom(chrom_train)

In [27]:
chronom_upd.head()

,NPLV,O2_SUM_CHR
0,510008,2909.0
1,510009,2182.0
2,510010,2841.0
3,510011,10.0
4,510012,3225.0


In [28]:
lom_upd=update_lom(lom_train)

In [29]:
lom_upd.head()

,NPLV,VDL_SUM,LOM_SUM
0,510008,25,76200
1,510009,25,78600
2,510010,48,76300
3,510011,25,84100
4,510012,25,76100


In [30]:
big_data=pd.merge(chronom_upd, lom_upd, on='NPLV', how='left')

In [31]:
big_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2063 entries, 0 to 2062
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   NPLV        2063 non-null   int64  
 1   O2_SUM_CHR  2063 non-null   float64
 2   VDL_SUM     2063 non-null   int64  
 3   LOM_SUM     2063 non-null   int64  
dtypes: float64(1), int64(3)
memory usage: 80.6 KB


In [32]:
big_data=pd.merge(big_data, chugun_train,on='NPLV', how='left')

In [33]:
big_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2063 entries, 0 to 2062
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   NPLV         2063 non-null   int64         
 1   O2_SUM_CHR   2063 non-null   float64       
 2   VDL_SUM      2063 non-null   int64         
 3   LOM_SUM      2063 non-null   int64         
 4   VES          2063 non-null   float64       
 5   T            2063 non-null   float64       
 6   SI           2063 non-null   float64       
 7   MN           2063 non-null   float64       
 8   S            2063 non-null   float64       
 9   P            2063 non-null   float64       
 10  CR           2063 non-null   float64       
 11  NI           2063 non-null   float64       
 12  CU           2063 non-null   float64       
 13  V            2063 non-null   float64       
 14  TI           2063 non-null   float64       
 15  DATA_ZAMERA  2063 non-null   datetime64[ns]
dtypes: dat

In [34]:
plavki_train[plavki_train['NPLV']==511135]

,NPLV,plavka_VR_NACH,plavka_VR_KON,plavka_NMZ,plavka_NAPR_ZAD,plavka_STFUT,plavka_TIPE_FUR,plavka_ST_FURM,plavka_TIPE_GOL,plavka_ST_GOL
1007,511135,2021-02-27 20:27:36,2021-02-27 21:28:44,С255,МНЛЗ,2098,цилиндрическая,63,5 сопловая,161
1008,511135,2021-02-27 20:27:36,2021-02-27 21:28:44,С255,МНЛЗ,2098,цилиндрическая,64,5 сопловая,162
1009,511135,2021-02-27 20:27:36,2021-02-27 21:28:44,С255,МНЛЗ,2098,цилиндрическая,65,5 сопловая,163
1010,511135,2021-02-27 20:27:36,2021-02-27 21:28:44,С255,МНЛЗ,2098,цилиндрическая,66,5 сопловая,164
1011,511135,2021-02-27 20:27:36,2021-02-27 21:28:44,С255,МНЛЗ,2098,цилиндрическая,67,5 сопловая,165
...,...,...,...,...,...,...,...,...,...,...
1077,511135,2021-02-27 20:27:36,2021-02-27 21:28:44,С255,МНЛЗ,2098,цилиндрическая,42,5 сопловая,42
1078,511135,2021-02-27 20:27:36,2021-02-27 21:28:44,С255,МНЛЗ,2098,цилиндрическая,43,5 сопловая,43
1079,511135,2021-02-27 20:27:36,2021-02-27 21:28:44,С255,МНЛЗ,2098,цилиндрическая,44,5 сопловая,44
1080,511135,2021-02-27 20:27:36,2021-02-27 21:28:44,С255,МНЛЗ,2098,цилиндрическая,45,5 сопловая,45


In [35]:
plavki_train=plavki_train.drop_duplicates('NPLV')

In [36]:
big_data=pd.merge(big_data, plavki_train,on='NPLV', how='left')

In [37]:
big_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2063 entries, 0 to 2062
Data columns (total 25 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   NPLV             2063 non-null   int64         
 1   O2_SUM_CHR       2063 non-null   float64       
 2   VDL_SUM          2063 non-null   int64         
 3   LOM_SUM          2063 non-null   int64         
 4   VES              2063 non-null   float64       
 5   T                2063 non-null   float64       
 6   SI               2063 non-null   float64       
 7   MN               2063 non-null   float64       
 8   S                2063 non-null   float64       
 9   P                2063 non-null   float64       
 10  CR               2063 non-null   float64       
 11  NI               2063 non-null   float64       
 12  CU               2063 non-null   float64       
 13  V                2063 non-null   float64       
 14  TI               2063 non-null   float64

In [38]:
big_data=pd.merge(big_data, produv_upd,on='NPLV', how='left')

In [39]:
big_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2063 entries, 0 to 2062
Data columns (total 32 columns):
 #   Column             Non-Null Count  Dtype          
---  ------             --------------  -----          
 0   NPLV               2063 non-null   int64          
 1   O2_SUM_CHR         2063 non-null   float64        
 2   VDL_SUM            2063 non-null   int64          
 3   LOM_SUM            2063 non-null   int64          
 4   VES                2063 non-null   float64        
 5   T                  2063 non-null   float64        
 6   SI                 2063 non-null   float64        
 7   MN                 2063 non-null   float64        
 8   S                  2063 non-null   float64        
 9   P                  2063 non-null   float64        
 10  CR                 2063 non-null   float64        
 11  NI                 2063 non-null   float64        
 12  CU                 2063 non-null   float64        
 13  V                  2063 non-null   float64      

In [40]:
big_data=pd.merge(big_data, gas_upd,on='NPLV', how='left')

In [41]:
big_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2063 entries, 0 to 2062
Data columns (total 52 columns):
 #   Column              Non-Null Count  Dtype          
---  ------              --------------  -----          
 0   NPLV                2063 non-null   int64          
 1   O2_SUM_CHR          2063 non-null   float64        
 2   VDL_SUM             2063 non-null   int64          
 3   LOM_SUM             2063 non-null   int64          
 4   VES                 2063 non-null   float64        
 5   T                   2063 non-null   float64        
 6   SI                  2063 non-null   float64        
 7   MN                  2063 non-null   float64        
 8   S                   2063 non-null   float64        
 9   P                   2063 non-null   float64        
 10  CR                  2063 non-null   float64        
 11  NI                  2063 non-null   float64        
 12  CU                  2063 non-null   float64        
 13  V                   2063 non-null

In [42]:
gas_upd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2063 entries, 0 to 2062
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   NPLV                2063 non-null   int64         
 1   GAS_START           2063 non-null   datetime64[ns]
 2   V_START             2063 non-null   float64       
 3   V_FINISH            2063 non-null   float64       
 4   T_START             2063 non-null   float64       
 5   T_FINISH            2063 non-null   float64       
 6   O2_SUM              2063 non-null   float64       
 7   N2_SUM              2063 non-null   float64       
 8   H2_SUM              2063 non-null   float64       
 9   CO2_SUM             2063 non-null   float64       
 10  CO_SUM              2063 non-null   float64       
 11  AR_SUM              2063 non-null   float64       
 12  TF1_START           2063 non-null   float64       
 13  TF1_FINISH          2063 non-null   float64     

In [43]:
big_data=pd.merge(big_data, sip_upd,on='NPLV', how='left')

In [44]:
big_data

,NPLV,O2_SUM_CHR,VDL_SUM,LOM_SUM,VES,T,SI,MN,S,P,...,"GAS_TIME,SEC",TOTAL_104,TOTAL_119,TOTAL_171,TOTAL_346,TOTAL_397,TOTAL_408,TOTAL_442,SIP_START,"SIP_TIME,SEC"
0,510008,2909.0,25,76200,263700.0,1396.0,0.44,0.22,0.023,0.097,...,2579,NaN,NaN,980.0,2950.0,NaN,14080.0,2960.0,2021-01-01 03:03:53,1795
1,510009,2182.0,25,78600,264500.0,1419.0,0.68,0.20,0.017,0.087,...,4002,1060.0,NaN,960.0,2930.0,NaN,18830.0,NaN,2021-01-01 03:55:43,2670
2,510010,2841.0,48,76300,263800.0,1384.0,0.56,0.26,0.017,0.096,...,2903,990.0,NaN,1050.0,2990.0,NaN,16080.0,2960.0,2021-01-01 05:09:20,1892
3,510011,10.0,25,84100,264000.0,1401.0,0.48,0.27,0.018,0.091,...,3291,550.0,NaN,980.0,3620.0,NaN,22150.0,NaN,2021-01-01 06:01:36,3304
4,510012,3225.0,25,76100,263300.0,1422.0,0.47,0.23,0.018,0.096,...,2894,2050.0,NaN,1000.0,2930.0,NaN,19550.0,3010.0,2021-01-01 07:11:44,2315
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2058,512318,1917.0,96,73600,267200.0,1415.0,0.38,0.28,0.019,0.099,...,3084,NaN,NaN,NaN,NaN,NaN,14640.0,2890.0,2021-04-26 12:58:37,2815
2059,512319,1660.0,48,76600,266800.0,1405.0,0.50,0.30,0.017,0.104,...,3842,NaN,NaN,NaN,960.0,NaN,19210.0,2820.0,2021-04-26 13:57:24,3947
2060,512320,0.0,65,64200,276100.0,1398.0,0.61,0.31,0.025,0.115,...,3305,NaN,NaN,NaN,1380.0,NaN,19420.0,2830.0,2021-04-26 15:17:30,2905
2061,512321,371.0,25,66200,275800.0,1408.0,0.38,0.27,0.021,0.100,...,3659,NaN,NaN,NaN,520.0,NaN,16100.0,2310.0,2021-04-26 16:19:13,2015


In [45]:
big_data.columns

Index(['NPLV', 'O2_SUM_CHR', 'VDL_SUM', 'LOM_SUM', 'VES', 'T', 'SI', 'MN', 'S',
       'P', 'CR', 'NI', 'CU', 'V', 'TI', 'DATA_ZAMERA', 'plavka_VR_NACH',
       'plavka_VR_KON', 'plavka_NMZ', 'plavka_NAPR_ZAD', 'plavka_STFUT',
       'plavka_TIPE_FUR', 'plavka_ST_FURM', 'plavka_TIPE_GOL', 'plavka_ST_GOL',
       'PRODUVKA_START', 'PRODUVKA_END', 'RASHOD_SUM', 'POL_MAX', 'POL_MIN',
       'PRODUVKA_TIME', 'PRODUVKA_TIME,SEC', 'GAS_START', 'V_START',
       'V_FINISH', 'T_START', 'T_FINISH', 'O2_SUM', 'N2_SUM', 'H2_SUM',
       'CO2_SUM', 'CO_SUM', 'AR_SUM', 'TF1_START', 'TF1_FINISH', 'TF2_START',
       'TF2_FINISH', 'O2_PRESSURE_START', 'O2_PRESSURE_FINISH',
       'O2_PRESSURE_MAX', 'O2_PRESSURE_MIN', 'GAS_TIME,SEC', 'TOTAL_104',
       'TOTAL_119', 'TOTAL_171', 'TOTAL_346', 'TOTAL_397', 'TOTAL_408',
       'TOTAL_442', 'SIP_START', 'SIP_TIME,SEC'],
      dtype='object')

In [46]:
big_data['DATA_ZAMERA']=pd.to_datetime(big_data['DATA_ZAMERA'], format='%Y-%m-%d %H:%M:%S')
big_data['plavka_VR_NACH']=pd.to_datetime(big_data['plavka_VR_NACH'], format='%Y-%m-%d %H:%M:%S')
big_data['plavka_VR_KON']=pd.to_datetime(big_data['plavka_VR_KON'], format='%Y-%m-%d %H:%M:%S')


big_data['CHUGUN_TIME']=big_data['DATA_ZAMERA']-big_data['plavka_VR_NACH']
big_data['CHUGUN_TIME,SEC']=big_data['CHUGUN_TIME'].dt.seconds

big_data['PLAVKA_TTL_TIME']=big_data['plavka_VR_KON']-big_data['plavka_VR_NACH']
big_data['PLAVKA_TTL_TIME,SEC']=big_data['PLAVKA_TTL_TIME'].dt.seconds

big_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2063 entries, 0 to 2062
Data columns (total 65 columns):
 #   Column               Non-Null Count  Dtype          
---  ------               --------------  -----          
 0   NPLV                 2063 non-null   int64          
 1   O2_SUM_CHR           2063 non-null   float64        
 2   VDL_SUM              2063 non-null   int64          
 3   LOM_SUM              2063 non-null   int64          
 4   VES                  2063 non-null   float64        
 5   T                    2063 non-null   float64        
 6   SI                   2063 non-null   float64        
 7   MN                   2063 non-null   float64        
 8   S                    2063 non-null   float64        
 9   P                    2063 non-null   float64        
 10  CR                   2063 non-null   float64        
 11  NI                   2063 non-null   float64        
 12  CU                   2063 non-null   float64        
 13  V                 

In [47]:
columns_to_drop=['DATA_ZAMERA',
                'plavka_VR_NACH',      
                 'plavka_VR_KON',  
                 'PRODUVKA_START' , 
                 'PRODUVKA_END' ,  
                 'PRODUVKA_TIME',
                 'GAS_START',
                 'SIP_START',
                 'PLAVKA_TTL_TIME',
                 'CHUGUN_TIME'  
                 
]

In [48]:
big_data=big_data.drop(columns_to_drop, axis=1)

In [49]:
big_data

,NPLV,O2_SUM_CHR,VDL_SUM,LOM_SUM,VES,T,SI,MN,S,P,...,TOTAL_104,TOTAL_119,TOTAL_171,TOTAL_346,TOTAL_397,TOTAL_408,TOTAL_442,"SIP_TIME,SEC","CHUGUN_TIME,SEC","PLAVKA_TTL_TIME,SEC"
0,510008,2909.0,25,76200,263700.0,1396.0,0.44,0.22,0.023,0.097,...,NaN,NaN,980.0,2950.0,NaN,14080.0,2960.0,1795,412,2579
1,510009,2182.0,25,78600,264500.0,1419.0,0.68,0.20,0.017,0.087,...,1060.0,NaN,960.0,2930.0,NaN,18830.0,NaN,2670,1384,4004
2,510010,2841.0,48,76300,263800.0,1384.0,0.56,0.26,0.017,0.096,...,990.0,NaN,1050.0,2990.0,NaN,16080.0,2960.0,1892,551,2904
3,510011,10.0,25,84100,264000.0,1401.0,0.48,0.27,0.018,0.091,...,550.0,NaN,980.0,3620.0,NaN,22150.0,NaN,3304,429,3291
4,510012,3225.0,25,76100,263300.0,1422.0,0.47,0.23,0.018,0.096,...,2050.0,NaN,1000.0,2930.0,NaN,19550.0,3010.0,2315,558,2895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2058,512318,1917.0,96,73600,267200.0,1415.0,0.38,0.28,0.019,0.099,...,NaN,NaN,NaN,NaN,NaN,14640.0,2890.0,2815,85754,3084
2059,512319,1660.0,48,76600,266800.0,1405.0,0.50,0.30,0.017,0.104,...,NaN,NaN,NaN,960.0,NaN,19210.0,2820.0,3947,697,3843
2060,512320,0.0,65,64200,276100.0,1398.0,0.61,0.31,0.025,0.115,...,NaN,NaN,NaN,1380.0,NaN,19420.0,2830.0,2905,83512,3305
2061,512321,371.0,25,66200,275800.0,1408.0,0.38,0.27,0.021,0.100,...,NaN,NaN,NaN,520.0,NaN,16100.0,2310.0,2015,84496,3660


In [50]:
big_data

,NPLV,O2_SUM_CHR,VDL_SUM,LOM_SUM,VES,T,SI,MN,S,P,...,TOTAL_104,TOTAL_119,TOTAL_171,TOTAL_346,TOTAL_397,TOTAL_408,TOTAL_442,"SIP_TIME,SEC","CHUGUN_TIME,SEC","PLAVKA_TTL_TIME,SEC"
0,510008,2909.0,25,76200,263700.0,1396.0,0.44,0.22,0.023,0.097,...,NaN,NaN,980.0,2950.0,NaN,14080.0,2960.0,1795,412,2579
1,510009,2182.0,25,78600,264500.0,1419.0,0.68,0.20,0.017,0.087,...,1060.0,NaN,960.0,2930.0,NaN,18830.0,NaN,2670,1384,4004
2,510010,2841.0,48,76300,263800.0,1384.0,0.56,0.26,0.017,0.096,...,990.0,NaN,1050.0,2990.0,NaN,16080.0,2960.0,1892,551,2904
3,510011,10.0,25,84100,264000.0,1401.0,0.48,0.27,0.018,0.091,...,550.0,NaN,980.0,3620.0,NaN,22150.0,NaN,3304,429,3291
4,510012,3225.0,25,76100,263300.0,1422.0,0.47,0.23,0.018,0.096,...,2050.0,NaN,1000.0,2930.0,NaN,19550.0,3010.0,2315,558,2895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2058,512318,1917.0,96,73600,267200.0,1415.0,0.38,0.28,0.019,0.099,...,NaN,NaN,NaN,NaN,NaN,14640.0,2890.0,2815,85754,3084
2059,512319,1660.0,48,76600,266800.0,1405.0,0.50,0.30,0.017,0.104,...,NaN,NaN,NaN,960.0,NaN,19210.0,2820.0,3947,697,3843
2060,512320,0.0,65,64200,276100.0,1398.0,0.61,0.31,0.025,0.115,...,NaN,NaN,NaN,1380.0,NaN,19420.0,2830.0,2905,83512,3305
2061,512321,371.0,25,66200,275800.0,1408.0,0.38,0.27,0.021,0.100,...,NaN,NaN,NaN,520.0,NaN,16100.0,2310.0,2015,84496,3660


In [51]:
big_data.columns

Index(['NPLV', 'O2_SUM_CHR', 'VDL_SUM', 'LOM_SUM', 'VES', 'T', 'SI', 'MN', 'S',
       'P', 'CR', 'NI', 'CU', 'V', 'TI', 'plavka_NMZ', 'plavka_NAPR_ZAD',
       'plavka_STFUT', 'plavka_TIPE_FUR', 'plavka_ST_FURM', 'plavka_TIPE_GOL',
       'plavka_ST_GOL', 'RASHOD_SUM', 'POL_MAX', 'POL_MIN',
       'PRODUVKA_TIME,SEC', 'V_START', 'V_FINISH', 'T_START', 'T_FINISH',
       'O2_SUM', 'N2_SUM', 'H2_SUM', 'CO2_SUM', 'CO_SUM', 'AR_SUM',
       'TF1_START', 'TF1_FINISH', 'TF2_START', 'TF2_FINISH',
       'O2_PRESSURE_START', 'O2_PRESSURE_FINISH', 'O2_PRESSURE_MAX',
       'O2_PRESSURE_MIN', 'GAS_TIME,SEC', 'TOTAL_104', 'TOTAL_119',
       'TOTAL_171', 'TOTAL_346', 'TOTAL_397', 'TOTAL_408', 'TOTAL_442',
       'SIP_TIME,SEC', 'CHUGUN_TIME,SEC', 'PLAVKA_TTL_TIME,SEC'],
      dtype='object')

In [52]:
big_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2063 entries, 0 to 2062
Data columns (total 55 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   NPLV                 2063 non-null   int64  
 1   O2_SUM_CHR           2063 non-null   float64
 2   VDL_SUM              2063 non-null   int64  
 3   LOM_SUM              2063 non-null   int64  
 4   VES                  2063 non-null   float64
 5   T                    2063 non-null   float64
 6   SI                   2063 non-null   float64
 7   MN                   2063 non-null   float64
 8   S                    2063 non-null   float64
 9   P                    2063 non-null   float64
 10  CR                   2063 non-null   float64
 11  NI                   2063 non-null   float64
 12  CU                   2063 non-null   float64
 13  V                    2063 non-null   float64
 14  TI                   2063 non-null   float64
 15  plavka_NMZ           2063 non-null   o

In [53]:
round(big_data.describe().T,1)

,count,mean,std,min,25%,50%,75%,max
NPLV,2063.0,511162.5,665.4,510008.0,510584.5,511163.0,511729.5,512322.0
O2_SUM_CHR,2063.0,2281.6,1247.7,0.0,1686.5,2196.0,3147.5,16429.0
VDL_SUM,2063.0,43.5,28.4,4.0,25.0,25.0,66.0,150.0
LOM_SUM,2063.0,69267.4,11230.3,17300.0,62400.0,70500.0,76800.0,155400.0
VES,2063.0,266529.8,12221.4,0.0,264400.0,266000.0,269700.0,312700.0
T,2063.0,1397.3,58.9,0.0,1385.0,1402.0,1417.0,1470.0
SI,2063.0,0.5,0.2,0.0,0.4,0.5,0.6,1.0
MN,2063.0,0.3,0.1,0.0,0.2,0.3,0.3,0.5
S,2063.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1
P,2063.0,0.1,0.0,0.0,0.1,0.1,0.1,0.1


In [54]:
big_data=pd.merge(big_data,target_train, how='left', on='NPLV')

In [55]:
big_data

,NPLV,O2_SUM_CHR,VDL_SUM,LOM_SUM,VES,T,SI,MN,S,P,...,TOTAL_171,TOTAL_346,TOTAL_397,TOTAL_408,TOTAL_442,"SIP_TIME,SEC","CHUGUN_TIME,SEC","PLAVKA_TTL_TIME,SEC",TST,C
0,510008,2909.0,25,76200,263700.0,1396.0,0.44,0.22,0.023,0.097,...,980.0,2950.0,NaN,14080.0,2960.0,1795,412,2579,1690,0.060
1,510009,2182.0,25,78600,264500.0,1419.0,0.68,0.20,0.017,0.087,...,960.0,2930.0,NaN,18830.0,NaN,2670,1384,4004,1683,0.097
2,510010,2841.0,48,76300,263800.0,1384.0,0.56,0.26,0.017,0.096,...,1050.0,2990.0,NaN,16080.0,2960.0,1892,551,2904,1662,0.091
3,510011,10.0,25,84100,264000.0,1401.0,0.48,0.27,0.018,0.091,...,980.0,3620.0,NaN,22150.0,NaN,3304,429,3291,1609,0.410
4,510012,3225.0,25,76100,263300.0,1422.0,0.47,0.23,0.018,0.096,...,1000.0,2930.0,NaN,19550.0,3010.0,2315,558,2895,1682,0.120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2058,512318,1917.0,96,73600,267200.0,1415.0,0.38,0.28,0.019,0.099,...,NaN,NaN,NaN,14640.0,2890.0,2815,85754,3084,1626,0.145
2059,512319,1660.0,48,76600,266800.0,1405.0,0.50,0.30,0.017,0.104,...,NaN,960.0,NaN,19210.0,2820.0,3947,697,3843,1643,0.087
2060,512320,0.0,65,64200,276100.0,1398.0,0.61,0.31,0.025,0.115,...,NaN,1380.0,NaN,19420.0,2830.0,2905,83512,3305,1615,0.141
2061,512321,371.0,25,66200,275800.0,1408.0,0.38,0.27,0.021,0.100,...,NaN,520.0,NaN,16100.0,2310.0,2015,84496,3660,1654,0.270


In [56]:
big_data['O2_SUM_CHR']=big_data.groupby(['TST'])['O2_SUM_CHR'].transform(lambda x: x.replace(0, int(x.mean())))

In [57]:
big_data=big_data.fillna(0)

In [58]:
big_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2063 entries, 0 to 2062
Data columns (total 57 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   NPLV                 2063 non-null   int64  
 1   O2_SUM_CHR           2063 non-null   float64
 2   VDL_SUM              2063 non-null   int64  
 3   LOM_SUM              2063 non-null   int64  
 4   VES                  2063 non-null   float64
 5   T                    2063 non-null   float64
 6   SI                   2063 non-null   float64
 7   MN                   2063 non-null   float64
 8   S                    2063 non-null   float64
 9   P                    2063 non-null   float64
 10  CR                   2063 non-null   float64
 11  NI                   2063 non-null   float64
 12  CU                   2063 non-null   float64
 13  V                    2063 non-null   float64
 14  TI                   2063 non-null   float64
 15  plavka_NMZ           2063 non-null   o

In [59]:
X_train=big_data.drop(['TST','C'], axis=1)
Y_train=big_data['TST']
Z_train=big_data['C']

In [60]:
X_train

,NPLV,O2_SUM_CHR,VDL_SUM,LOM_SUM,VES,T,SI,MN,S,P,...,TOTAL_104,TOTAL_119,TOTAL_171,TOTAL_346,TOTAL_397,TOTAL_408,TOTAL_442,"SIP_TIME,SEC","CHUGUN_TIME,SEC","PLAVKA_TTL_TIME,SEC"
0,510008,2909.0,25,76200,263700.0,1396.0,0.44,0.22,0.023,0.097,...,0.0,0.0,980.0,2950.0,0.0,14080.0,2960.0,1795,412,2579
1,510009,2182.0,25,78600,264500.0,1419.0,0.68,0.20,0.017,0.087,...,1060.0,0.0,960.0,2930.0,0.0,18830.0,0.0,2670,1384,4004
2,510010,2841.0,48,76300,263800.0,1384.0,0.56,0.26,0.017,0.096,...,990.0,0.0,1050.0,2990.0,0.0,16080.0,2960.0,1892,551,2904
3,510011,10.0,25,84100,264000.0,1401.0,0.48,0.27,0.018,0.091,...,550.0,0.0,980.0,3620.0,0.0,22150.0,0.0,3304,429,3291
4,510012,3225.0,25,76100,263300.0,1422.0,0.47,0.23,0.018,0.096,...,2050.0,0.0,1000.0,2930.0,0.0,19550.0,3010.0,2315,558,2895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2058,512318,1917.0,96,73600,267200.0,1415.0,0.38,0.28,0.019,0.099,...,0.0,0.0,0.0,0.0,0.0,14640.0,2890.0,2815,85754,3084
2059,512319,1660.0,48,76600,266800.0,1405.0,0.50,0.30,0.017,0.104,...,0.0,0.0,0.0,960.0,0.0,19210.0,2820.0,3947,697,3843
2060,512320,2968.0,65,64200,276100.0,1398.0,0.61,0.31,0.025,0.115,...,0.0,0.0,0.0,1380.0,0.0,19420.0,2830.0,2905,83512,3305
2061,512321,371.0,25,66200,275800.0,1408.0,0.38,0.27,0.021,0.100,...,0.0,0.0,0.0,520.0,0.0,16100.0,2310.0,2015,84496,3660


In [61]:
Y_train

0       1690
1       1683
2       1662
3       1609
4       1682
        ... 
2058    1626
2059    1643
2060    1615
2061    1654
2062    1630
Name: TST, Length: 2063, dtype: int64

In [62]:
Z_train

0       0.060
1       0.097
2       0.091
3       0.410
4       0.120
        ...  
2058    0.145
2059    0.087
2060    0.141
2061    0.270
2062    0.183
Name: C, Length: 2063, dtype: float64

In [63]:
X = big_data.drop(['TST','C'], axis=1)
y = big_data.pop('TST')

# Label encoding for categoricals
for colname in X.select_dtypes("object"):
    X[colname], _ = X[colname].factorize()

# All discrete features should now have integer dtypes (double-check this before using MI!)
discrete_features = X.dtypes == int

In [64]:
from sklearn.feature_selection import mutual_info_regression

def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

mi_scores = make_mi_scores(X, y, discrete_features)
mi_scores[::3]  # show a few features with their MI scores

O2_SUM_CHR             0.465758
plavka_NAPR_ZAD        0.039669
V                      0.036587
O2_PRESSURE_MIN        0.027183
LOM_SUM                0.020325
TF2_FINISH             0.016707
TF1_START              0.014697
TI                     0.011599
CU                     0.010539
PLAVKA_TTL_TIME,SEC    0.006960
GAS_TIME,SEC           0.005302
SI                     0.002122
CR                     0.001274
TOTAL_442              0.000000
TOTAL_104              0.000000
PRODUVKA_TIME,SEC      0.000000
T_FINISH               0.000000
T_START                0.000000
V_FINISH               0.000000
Name: MI Scores, dtype: float64

In [65]:
z=big_data.pop('C')

In [66]:
mi_scores = make_mi_scores(X, z, discrete_features)
mi_scores[::3]  # show a few features with their MI scores

plavka_NMZ           0.132051
NPLV                 0.114798
CO_SUM               0.062236
plavka_ST_FURM       0.052637
O2_SUM               0.043949
GAS_TIME,SEC         0.038469
TF1_FINISH           0.034521
V_FINISH             0.028702
P                    0.022703
CU                   0.018092
T                    0.017176
TI                   0.013296
NI                   0.008392
O2_PRESSURE_START    0.006673
V                    0.004871
T_FINISH             0.002860
TOTAL_119            0.000000
TOTAL_346            0.000000
VDL_SUM              0.000000
Name: MI Scores, dtype: float64

In [67]:
from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.65, random_state=12345)

In [68]:
categorical_features_indices = np.where(big_data.dtypes=='object')[0]

In [69]:
categorical_features_indices

array([15, 16, 18, 20], dtype=int64)

In [70]:
from catboost import CatBoostRegressor

model=CatBoostRegressor(iterations=100, depth=10, learning_rate=0.1, loss_function='RMSE')
model.fit(X_train, y_train,cat_features=categorical_features_indices,eval_set=(X_validation, y_validation),plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 30.1878654	test: 29.4409863	best: 29.4409863 (0)	total: 257ms	remaining: 25.4s
1:	learn: 29.4424203	test: 29.0538137	best: 29.0538137 (1)	total: 372ms	remaining: 18.3s
2:	learn: 28.7542777	test: 28.7159775	best: 28.7159775 (2)	total: 488ms	remaining: 15.8s
3:	learn: 28.1054032	test: 28.3626997	best: 28.3626997 (3)	total: 607ms	remaining: 14.6s
4:	learn: 27.4589619	test: 28.0126356	best: 28.0126356 (4)	total: 725ms	remaining: 13.8s
5:	learn: 26.9244806	test: 27.6166438	best: 27.6166438 (5)	total: 841ms	remaining: 13.2s
6:	learn: 26.3185412	test: 27.2032420	best: 27.2032420 (6)	total: 961ms	remaining: 12.8s
7:	learn: 25.7617870	test: 26.8792666	best: 26.8792666 (7)	total: 1.08s	remaining: 12.4s
8:	learn: 25.2609607	test: 26.7622188	best: 26.7622188 (8)	total: 1.19s	remaining: 12s
9:	learn: 24.8075438	test: 26.6099477	best: 26.6099477 (9)	total: 1.29s	remaining: 11.7s
10:	learn: 24.3635336	test: 26.3995377	best: 26.3995377 (10)	total: 1.4s	remaining: 11.3s
11:	learn: 23.9556752	

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid={'iterations':range(50,550,50),
            'learning_rate':[0.1,0.01,0.5,0.05,0.0001],
            'loss_function':['RMSE'],
            'l2_leaf_reg': [1, 3, 5, 7, 9,25],
            'depth':range(5,50,1)
            }

clf = GridSearchCV(estimator=model,param_grid=param_grid)
clf.fit(X_train, y_train)


0:	learn: 30.7362678	total: 2.92ms	remaining: 143ms
1:	learn: 29.8618374	total: 5.74ms	remaining: 138ms
2:	learn: 29.2850699	total: 8.61ms	remaining: 135ms
3:	learn: 28.7829208	total: 10.9ms	remaining: 125ms
4:	learn: 28.1904950	total: 13.2ms	remaining: 119ms
5:	learn: 27.7887873	total: 15.6ms	remaining: 115ms
6:	learn: 27.4150437	total: 18ms	remaining: 110ms
7:	learn: 27.1027607	total: 20.2ms	remaining: 106ms
8:	learn: 26.7021171	total: 22.6ms	remaining: 103ms
9:	learn: 26.4241087	total: 24.8ms	remaining: 99.3ms
10:	learn: 26.1253978	total: 27.1ms	remaining: 96ms
11:	learn: 25.8882455	total: 29.8ms	remaining: 94.3ms
12:	learn: 25.7268263	total: 31.9ms	remaining: 90.7ms
13:	learn: 25.5003872	total: 34.1ms	remaining: 87.7ms
14:	learn: 25.3744415	total: 36.3ms	remaining: 84.8ms
15:	learn: 25.1658873	total: 38.5ms	remaining: 81.8ms
16:	learn: 24.9467658	total: 40.7ms	remaining: 79.1ms
17:	learn: 24.6980922	total: 42.9ms	remaining: 76.3ms
18:	learn: 24.5887101	total: 45.2ms	remaining: 73.7